## Configuration

In [ ]:
source ./config.sh

In [ ]:
export ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
export BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

## Create clusters

In [ ]:
tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${CICD_CLUSTER}
tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${ALPHA_CLUSTER}
tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 2 --instance-type m5.xlarge --group ${GROUP} --name ${BRAVO_CLUSTER}

In [ ]:
watch -d -n 5 tmc cluster list --group bush

## Add clusters to kubeconfig file

In [ ]:
KUBECONFIG_CICD=`mktemp`
tmc cluster provisionedcluster kubeconfig get ${CICD_CLUSTER} >> ${KUBECONFIG_CICD}

KUBECONFIG_ALPHA=`mktemp`
tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} >> ${KUBECONFIG_ALPHA}

KUBECONFIG_BRAVO=`mktemp`
tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} >> ${KUBECONFIG_BRAVO}

KUBECONFIG=${KUBECONFIG_CICD}:${KUBECONFIG_ALPHA}:${KUBECONFIG_BRAVO} kubectl config view --merge --flatten > ~/.kube/config

rm ${KUBECONFIG_CICD}
rm ${KUBECONFIG_ALPHA}
rm ${KUBECONFIG_BRAVO}

## Update cluster permissions

In [ ]:
kubectl config use-context ${CICD_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

echo
kubectl config use-context ${ALPHA_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

kubectl create -n kubeapps serviceaccount kubeapps-operator
kubectl create clusterrolebinding kubeapps-operator --clusterrole=cluster-admin --serviceaccount=kubeapps:kubeapps-operator

## kubeapps permissions

In [ ]:
echo
echo 'Token for kubeapps on Alpha cluster:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

echo
kubectl create -n kubeapps serviceaccount kubeapps-operator
kubectl create clusterrolebinding kubeapps-operator --clusterrole=cluster-admin --serviceaccount=kubeapps:kubeapps-operator

echo
echo 'Token for kubeapps on Bravo cluster:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

## Enable automatic sidecar injection

In [ ]:
kubectl config use-context ${ALPHA_CLUSTER}
kubectl label namespace default istio-injection=enabled --overwrite

kubectl config use-context ${BRAVO_CLUSTER}
kubectl label namespace default istio-injection=enabled --overwrite

## Create namespaces

In [ ]:
kubectl config use-context ${ALPHA_CLUSTER}

kubectl create namespace wavefront
kubectl create namespace rabbit
kubectl create namespace kubeapps

kubectl config use-context ${BRAVO_CLUSTER}

kubectl create namespace wavefront
kubectl create namespace rabbit
kubectl create namespace kubeapps

## Delete any existing Docker Hub images

In [ ]:
set -e

TOKEN=$(curl -s -H "Content-Type: application/json" -X POST -d '{"username": "'${DOCKERHUB_USERNAME}'", "password": "'${DOCKERHUB_PASSWORD}'"}' https://hub.docker.com/v2/users/login/ | jq -r .token)

REPOSITORY_NAME="account-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="confirmation-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="payment-service"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

REPOSITORY_NAME="dotnet-core-react-example"
curl -X DELETE -s -H "Authorization: JWT ${TOKEN}" https://hub.docker.com/v2/repositories/${DOCKERHUB_USERNAME}/${REPOSITORY_NAME}/

### Configure Tanzu Service Mesh
- Onboard alpha & bravo clusters into Tanzu Service Mesh
- Create the "acme.com" global namespace and connect the default namespaces in the alpha & bravo clusters

## Install kpack

In [ ]:
kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Install kpack...'
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/01-kpack-release-0.0.9.yml

echo
echo 'Set the Github and Docker secrets...'
cat ./kubernetes/cicd/kpack/02-github-creds.template.yml | sed "s/GITHUB_USERNAME/$GITHUB_USERNAME/" | sed "s/GITHUB_PASSWORD/$GITHUB_PASSWORD/" | kubectl create -n kpack -f -
cat ./kubernetes/cicd/kpack/03-dockerhub-creds.template.yml | sed "s/DOCKERHUB_USERNAME/$DOCKERHUB_USERNAME/" | sed "s/DOCKERHUB_PASSWORD/$DOCKERHUB_PASSWORD/" |kubectl create -n kpack -f -

echo
echo 'Install the default kpack builder...'
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/04-default-builder.yml

echo
echo 'Configure the application images...'
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/account-image.yml
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/confirmation-image.yml
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/payment-image.yml
kubectl apply -n kpack -f ./kubernetes/cicd/kpack/images/dotnet-image.yml


## Install Argo CD

In [ ]:
kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Create Argo CD namespace...'
kubectl create namespace argocd

echo
echo 'Install Argo CD on the CI/CD cluster...'
kubectl -n argocd apply -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml

echo
echo 'Fetch a Load Balanced IP for the Argo CD server...'
kubectl -n argocd patch svc argocd-server -p '{"spec": {"type": "LoadBalancer"}}'

### Wait for load balancer to be provisioned...

In [ ]:
kubectl config use-context ${CICD_CLUSTER}

watch -d -n 5 "kubectl -n argocd get svc argocd-server && nslookup `kubectl -n argocd get svc/argocd-server -o json | jq '.status.loadBalancer.ingress[0].hostname' -j`"

In [ ]:
kubectl config use-context ${CICD_CLUSTER}

ARGOCD_USERNAME=admin
ARGOCD_PASSWORD=`kubectl -n argocd get pods -l app.kubernetes.io/name=argocd-server -o name | cut -d'/' -f 2`

echo
echo 'Store the Argo CD host name...'
ARGOCD_HOST=`kubectl -n argocd get svc/argocd-server -o json | jq '.status.loadBalancer.ingress[0].hostname' -j`

echo
echo "Argo CD Host: $ARGOCD_HOST"
echo "Username: $ARGOCD_USERNAME"
echo "Password: $ARGOCD_PASSWORD"

## Configure credentials

In [ ]:
kubectl config use-context ${ALPHA_CLUSTER}

echo
cat ./kubernetes/cicd/argocd/rabbit-secret.template.yml | sed "s/RABBITMQ_PASSWORD/$ALPHA_RABBITMQ_PASSWORD/"| kubectl create -n argocd -f -

cat ./kubernetes/cicd/argocd/rabbit-secret.template.yml | sed "s/RABBITMQ_PASSWORD/$ALPHA_RABBITMQ_PASSWORD/" | kubectl create -n rabbit -f -

cat ./kubernetes/cicd/argocd/wavefront-token.template.yml | sed "s/WAVEFRONT_TOKEN/$ALPHA_WAVEFRONT_TOKEN/" | kubectl create -n wavefront -f -

echo
kubectl config use-context ${BRAVO_CLUSTER}

cat ./kubernetes/cicd/argocd/wavefront-token.template.yml | sed "s/WAVEFRONT_TOKEN/$BRAVO_WAVEFRONT_TOKEN/" | kubectl create -n wavefront -f -

## Configure GitOps

In [ ]:
kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the projects in Argo CD...'
cat ./kubernetes/cicd/argocd/appproject-alpha.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl create -n argocd -f -
cat ./kubernetes/cicd/argocd/appproject-bravo.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl create -n argocd -f -

echo
echo 'Configure the Kubeapps Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/kubeapps.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl create -n argocd -f -
cat ./kubernetes/cicd/argocd/kubeapps.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl create -n argocd -f -

echo
echo 'Configure the RabbitMQ Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/rabbit.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl create -n argocd -f -

echo
echo 'Configure the Redis Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/redis.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl create -n argocd -f -
echo ''

echo
echo 'Configure the Wavefront Proxy Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/wavefront-proxy.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl create -n argocd -f -
cat ./kubernetes/cicd/argocd/wavefront-proxy.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl create -n argocd -f -

echo
echo 'Configure the applications in Argo CD...'
cat ./kubernetes/cicd/argocd/account-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl create -n argocd -f -
cat ./kubernetes/cicd/argocd/confirmation-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl create -n argocd -f -
cat ./kubernetes/cicd/argocd/payment-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl create -n argocd -f -
cat ./kubernetes/cicd/argocd/dotnet-service.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl create -n argocd -f -

## Retrieve the external hostname for the Istio ingress gateway

In [ ]:
kubectl config use-context ${ALPHA_CLUSTER}
kubectl get svc istio-ingressgateway -n istio-system

kubectl config use-context ${BRAVO_CLUSTER}
kubectl get svc istio-ingressgateway -n istio-system

## Delete clusters

In [ ]:
tmc cluster delete ${CICD_CLUSTER}
tmc cluster delete ${ALPHA_CLUSTER}
tmc cluster delete ${BRAVO_CLUSTER}

### To Do
- Add links to running applications
- Add links to TSM/TMC/TAC
- Rework the readme file
- Remove old scripts
- Use Harbor registry to demonstrate CVEs in public images